# Heart Disease Detection AI Model
This is a notebook describes the process and steps taken to train an algorithm to be able to detect whether a patient has heart disease or not.  
The algorithm that will be used is the __Random Forest Classifier algorithm__

> Steps being followed;
> 1. Get the data
> 2. Clean and transform the data (including encoding and scaling)
> 3. Fit the model
> 4. Test the model
> 5. Evaluate the model
> 6. Export the model

In [1]:
#import necessary packages that will be used
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

I'm making use of the heart disease dataset from https://github.com/mrdbourke/zero-to-mastery-ml/blob/master/data/heart-disease.csv

In [2]:
heart_disease_raw_ds = pd.read_csv("./data/heart-disease.csv")
heart_disease_raw_ds.head(7)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1
5,57,1,0,140,192,0,1,148,0,0.4,1,0,1,1
6,56,0,1,140,294,0,0,153,0,1.3,1,0,2,1


### Data Cleaning/Processing

**i. Handle missing vaues**

In [4]:
#checking for issin values
heart_disease_raw_ds.isna().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

From the check above there are no missing values in the dataset.  
The dataset has only numerical features, hence, there is no case of categorical features to encode

**ii Seperate the test data from the training data**

In [7]:
#first, seperate the features (X) and the target (Y)
x = heart_disease_raw_ds.drop("target", axis=1)
y = heart_disease_raw_ds["target"] #becuase its the target variable is the result it should be put on the y variable

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2, random_state=20) #the random state is defined to enable us do a reproducable outcome everytime (similar to defining the seed of a numpy random function)

**ii. Feature scaling using normalisation**

In [ ]:
#import the scaler and column transformer from scikitlearn
from sklearn.preprocessing import MinMaxScaler #this performs normalisation
from sklearn.compose import ColumnTransformer

In [19]:
min_max_scaler = MinMaxScaler()
features = x.keys() #we are applying scaling to all the determinnant features

features_transformer = ColumnTransformer([("features_scaling", min_max_scaler, features)], remainder="passthrough") #transform the columns

In [31]:
#now transform the x_train features
x_train_transformed = features_transformer.fit_transform(x_train)

x_train_transformed[:5]#preview the transformed x_train features

array([[0.76744186, 1.        , 0.        , 0.29245283, 0.29223744,
        1.        , 0.5       , 0.74193548, 0.        , 0.03225806,
        0.5       , 0.5       , 1.        ],
       [0.6744186 , 1.        , 1.        , 0.48113208, 0.24429224,
        1.        , 0.        , 0.63709677, 0.        , 0.37096774,
        0.        , 0.        , 0.33333333],
       [0.74418605, 1.        , 0.        , 0.16981132, 0.19634703,
        0.        , 0.        , 0.49193548, 1.        , 0.01612903,
        1.        , 0.25      , 0.66666667],
       [0.13953488, 1.        , 1.        , 0.43396226, 0.16666667,
        0.        , 0.5       , 0.86290323, 1.        , 0.22580645,
        1.        , 0.        , 1.        ],
       [0.55813953, 0.        , 0.        , 0.71698113, 0.2260274 ,
        1.        , 0.        , 0.60483871, 1.        , 0.4516129 ,
        0.5       , 0.5       , 0.33333333]])

In [32]:
#transform the x_test data too
x_test_transformed = features_transformer.fit_transform(x_test)
x_test_transformed[:5] #preview the transformed

array([[0.8       , 1.        , 1.        , 0.475     , 0.51086957,
        1.        , 0.        , 0.75438596, 0.        , 0.35      ,
        0.5       , 0.25      , 0.5       ],
       [0.6       , 1.        , 1.        , 0.25      , 0.1884058 ,
        0.        , 0.        , 0.64912281, 0.        , 0.475     ,
        0.5       , 0.        , 1.        ],
       [0.44444444, 1.        , 0.66666667, 0.25      , 0.17028986,
        0.        , 1.        , 0.44736842, 0.        , 0.5       ,
        0.5       , 0.75      , 1.        ],
       [0.66666667, 1.        , 0.33333333, 0.5       , 0.28985507,
        0.        , 1.        , 0.66666667, 1.        , 0.        ,
        1.        , 0.        , 0.5       ],
       [0.8       , 0.        , 0.66666667, 0.5       , 1.        ,
        1.        , 0.        , 0.60526316, 0.        , 0.2       ,
        1.        , 0.25      , 0.5       ]])

### Model Fitting
> Including hyperparameter tuning

In [42]:
prediction_model = RandomForestClassifier(n_estimators=100, random_state=20) #lets tune the "random_state" hyper parameter to 20 since the samples were created with the same random state. For the "n_estimators" hyper parameter, it is by default 100 (n_estimators influce the accuracy of models (in some cases, the smaller the number, the more accurate it is (depending on the context)))
prediction_model.fit(x_train_transformed, y_train) #fit the model with the transformed features
model_prediction = prediction_model.predict(x_test_transformed) #test the model


In [43]:
#check the prediction
model_prediction[:5]

array([1, 0, 0, 1, 1])

In [ ]:
#lets check the accuracy at a glance from the source table by merging its result to that of the y_test data
model_pred = pd.DataFrame({
    "model_prediction": model_prediction
})

#merged all the DataFrames together and made sure the indices are reset (becuase we sampled randomly when splitting the train-test data) so that when they're compiled, there won't be any mismatch of rows or NaNs because of randomised indices
result_table = pd.concat([x_test.reset_index(drop=True),y_test.reset_index(drop=True),model_pred], axis=1)
result_table.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,model_prediction
0,65,1,3,138,282,1,0,174,0,1.4,1,1,2,0,1
1,56,1,3,120,193,0,0,162,0,1.9,1,0,3,1,0
2,49,1,2,120,188,0,1,139,0,2.0,1,3,3,0,0
3,59,1,1,140,221,0,1,164,1,0.0,2,0,2,1,1
4,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1,1


### Model evaluation

In [47]:
#quick evaluation
prediction_model.score(x_test_transformed, y_test)

0.6885245901639344

**Evaluation using SciKit Learn metrics functions**

In [48]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [50]:
print(classification_report(y_test, model_prediction))

              precision    recall  f1-score   support

           0       0.56      0.78      0.65        23
           1       0.83      0.63      0.72        38

    accuracy                           0.69        61
   macro avg       0.70      0.71      0.69        61
weighted avg       0.73      0.69      0.69        61



In [51]:
print(confusion_matrix(y_test, model_prediction))

[[18  5]
 [14 24]]


In [52]:
print(accuracy_score(y_test, model_prediction))

0.6885245901639344


From the evaluation above, there is a lot of deviatons from the true target value as the algorithm is only 68% accurate

### Export the Model

In [53]:
import pickle

In [54]:
pickle.dump(prediction_model, open("./exported model/heart_disease_identifier_model.pkl", "wb")) #save the model by writing to the a pickle file. The "wb" parameter of the opn() function means "write binary". When you're loading a mdoel you use "rb" meaning "read binary"

I will either further tune the AI model or try another classifier model on this problem till the best result is attained